In [43]:
import os
import pandas as pd
# 第二步：递归遍历文件夹，读取并合并txt文件
folder_path = '/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA'
faers_data_type = ['demo', 'drug', 'reac', 'outc', 'ther', 'indi', 'rpsr']

def read_and_combine_files(directory):
    all_faers_data = {}

    for root, dirs, files in os.walk(directory):
        for filename in files:
            for item in faers_data_type:
                if filename.lower().endswith('.txt') and filename.lower().startswith(item):
                    file_path = os.path.join(root, filename)
                    # 如果随机数已经在字典中，则追加当前时间到对应的列表
                    if item in all_faers_data:
                        all_faers_data[item].append(file_path)
                    # 否则，为这个新的随机数创建一个包含当前时间的新列表
                    else:
                        all_faers_data[item] = []
                        all_faers_data[item].append(file_path)

    return all_faers_data

all_faers_data = read_and_combine_files(folder_path)
all_faers_data

{'reac': ['/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA/faers_ascii_2017Q2/ascii/REAC17Q2.txt',
  '/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA/faers_ascii_2019Q4/ascii/REAC19Q4.txt',
  '/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA/faers_ascii_2019Q3/ascii/REAC19Q3.txt',
  '/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA/faers_ascii_2017Q4/ascii/REAC17Q4.txt',
  '/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA/faers_ascii_2017Q3/ascii/REAC17Q3.txt',
  '/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA/faers_ascii_2019Q2/ascii/REAC19Q2.txt',
  '/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA/faers_ascii_2024Q1/ASCII/REAC24Q1.txt',
  '/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA/faers_ascii_2022Q3/ASCII/REAC22Q3.txt',
  '/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA/faers_ascii_2022Q4/ASCII/REAC22Q4.txt',
  '/Users/liuhaoran/PycharmProjects/Faers/data/FAERS_DATA/faers_ascii_2020Q1/ASCII/REAC20Q1.txt',
  '/Users/li

##  建表语句


In [20]:
create_table_sql = {}
create_table_sql['drug'] = '''
DROP TABLE faers_drug;
CREATE TABLE IF EXISTS faers_drug (
    PRIMARYID VARCHAR(255) NOT NULL,
    CASEID VARCHAR(255) NOT NULL,
    DRUG_SEQ VARCHAR(255) NOT NULL,
    ROLE_COD VARCHAR(255),
    DRUGNAME TEXT NOT NULL,
    PROD_AI TEXT,
    VAL_VBM VARCHAR(255),
    ROUTE VARCHAR(255),
    DOSE_VBM TEXT,
    CUM_DOSE_CHR TEXT,
    CUM_DOSE_UNIT VARCHAR(255),
    DECHAL VARCHAR(255),
    RECHAL VARCHAR(255),
    LOT_NUM TEXT,
    EXP_DT VARCHAR(255),
    NDA_NUM VARCHAR(255),
    DOSE_AMT TEXT,
    DOSE_UNIT VARCHAR(255),
    DOSE_FORM TEXT,
    DOSE_FREQ VARCHAR(255),
    PRIMARY KEY (PRIMARYID, CASEID, DRUG_SEQ)
);
'''

In [21]:
print(create_table_sql['drug'])



DROP TABLE faers_drug;
CREATE TABLE faers_drug (
    PRIMARYID VARCHAR(255) NOT NULL,
    CASEID VARCHAR(255) NOT NULL,
    DRUG_SEQ VARCHAR(255) NOT NULL,
    ROLE_COD VARCHAR(255),
    DRUGNAME TEXT NOT NULL,
    PROD_AI TEXT,
    VAL_VBM VARCHAR(255),
    ROUTE VARCHAR(255),
    DOSE_VBM TEXT,
    CUM_DOSE_CHR TEXT,
    CUM_DOSE_UNIT VARCHAR(255),
    DECHAL VARCHAR(255),
    RECHAL VARCHAR(255),
    LOT_NUM TEXT,
    EXP_DT VARCHAR(255),
    NDA_NUM VARCHAR(255),
    DOSE_AMT TEXT,
    DOSE_UNIT VARCHAR(255),
    DOSE_FORM TEXT,
    DOSE_FREQ VARCHAR(255),
    PRIMARY KEY (PRIMARYID, CASEID, DRUG_SEQ)
);



In [65]:
import os;
import sys;
## 每个数据库开始的部分
import_data_sql = {}
shell_cmd = {}
 
### sql 代码写入 import_data.sql
for table_name in ['drug',]: 
    ## 分别记录sql/sh代码
    import_data_sql[table_name] = '';
    shell_cmd[table_name] = '''export LC_ALL='C';'''
    
    import_data_sql[table_name] += '--  ||-----  BEGIN TO IMPORT DATABASE {} --- ;\n'.format(database_name);
    import_data_sql[table_name] += 'USE {};\n'.format(database_name);
    import_data_sql[table_name] += 'SET SQL_MODE=\'NO_BACKSLASH_ESCAPES\';\n';
    import_data_sql[table_name] += create_table_sql['drug'] + '\n';
    
    print('process for table {}'.format(table_name))
    for j in sorted(all_faers_data[table_name]):
        ## 处理 sed: illegal byte sequence 问题
        shell_cmd[table_name] += '''
                sed -ic \'s/\\\\n/SLASH_N_REPLACED/g\' {filename};\n
                sed -ic \'s/\\r/SLASH_R_REPLACED/g\' {filename}; \n 
            '''.format(filename=j)
        ## process_file inplace:substitue \n
        #os.system(cmd_replace_slash_n);
        
        ##### 导入数据  ###
        import_data_sql[table_name] += ''' LOAD DATA INFILE '{}' INTO TABLE {} FIELDS TERMINATED BY '$' ENCLOSED BY '"' LINES TERMINATED BY '\\n'  IGNORE 1 ROWS;\n\n'''.format(j, 'faers_' + table_name )
        ##### 导入数据  ###

    import_data_sql[table_name] += '-- --  FINSH IMPORTING DATABASE {} ----- ;\n'.format(database_name);
    shell_cmd[table_name] += '##  --  FINSH IMPORTING DATABASE {} ----- ;\n'.format(database_name);

    
    with open('import_data.sql', 'w') as f:
        #for i in all_faers_data: 
        f.write(import_data_sql[table_name]);       
  
    with open('import_data.sh', 'w') as f:
        f.write(shell_cmd[table_name]);   
      
#os.system('open import_data.sh')  
print('done')
            

process for table drug
done



## preporess `drug` table

In [68]:
import  os
folder_path='/Users/liuhaoran/Pictures'
image_extensions = ['.jpg', '.jpeg', '.png', '.gif']
img_list = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        if os.path.splitext(file_path)[1].lower() in image_extensions:
            img_list.append(file_path)
            
print(len(img_list))

40489
